In [1]:
ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [2]:
# Load FC

import pickle
import numpy as np

pncdir = '/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/PNC/'
lowrankdir = '/home/anton/Documents/Tulane/Research/Work/ContrastiveLearning/PNC/Top10/'
pncdemo = pickle.load(open(f'{pncdir}/demographics.pkl', 'rb'))
no_snps_subs = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/fc_subs_no_snps.pkl', 'rb'))

rest = []
nback = []
emoid = []
restmat = []
nbackmat = []
emoidmat = []
race = []
sex = []
age = []
subids = []

a,b = np.triu_indices(264,1)

for sub in pncdemo['age_at_cnb']:
    if sub in no_snps_subs:
        continue
    try:
        ra = pncdemo['Race'][sub]
        ag = pncdemo['age_at_cnb'][sub]
        se = pncdemo['Sex'][sub]
        if ra not in ['AA', 'EA']:
            continue
        ra = ra == 'AA'
        se = se == 'M'
        r = np.load(f'{pncdir}/fc/{sub}_task-rest_fc.npy')
        n = np.load(f'{pncdir}/fc/{sub}_task-nback_fc.npy')
        e = np.load(f'{pncdir}/fc/{sub}_task-emoid_fc.npy')
        r = remap(vec2mat(r))
        n = remap(vec2mat(n))
        e = remap(vec2mat(e))
        race.append(ra)
        sex.append(se)
        age.append(ag)
        rest.append(r[a,b])
        nback.append(n[a,b])
        emoid.append(e[a,b])
        subids.append(sub)
    except:
        pass

rest = np.stack(rest)
nback = np.stack(nback)
emoid = np.stack(emoid)
race = np.array(race).astype('int')
sex = np.array(sex).astype('int')
age = np.array(age)

age = (age-np.mean(age))/np.std(age)

print([a.shape for a in [rest, nback, emoid, race, sex, age]])

[(1155, 34716), (1155, 34716), (1155, 34716), (1155,), (1155,), (1155,)]


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

class VAE(nn.Module):
    def __init__(self, fcd, ld):
        super(VAE, self).__init__()
        self.fcd = fcd
        self.ld = ld
        self.enc1 = nn.Linear(fcd, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, ld).float().cuda()
        self.dec1 = nn.Linear(ld+6, 1000).float().cuda()
        self.dec2 = nn.Linear(1000, fcd).float().cuda()

    def enc(self, x):
        x = F.relu(self.enc1(x))
        z = self.enc2(x)
        return z

    def gen(self, n):
        return torch.randn(n, self.ld).float().cuda()/(10**0.5)
    
    def dec(self, z, age, sex, race, rest, nback, emoid):
        z = torch.cat([z, age.unsqueeze(1), sex.unsqueeze(1), race.unsqueeze(1), 
                       rest.unsqueeze(1), nback.unsqueeze(1), emoid.unsqueeze(1)], dim=1)
        x = F.relu(self.dec1(z))
        x = self.dec2(x)
        return x

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

vae = VAE(34716, 30)
vae.load_state_dict(torch.load('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/vae_1000_z30_cov6.torch'))
vae.eval()

print('Done')

Done


In [4]:
# Train MLP

class TaskSwitch(nn.Module):
    def __init__(self):
        super(TaskSwitch, self).__init__()
        self.fc1 = nn.Linear(34716, 1000).float().cuda()
        self.fc2 = nn.Linear(1000, 34716).float().cuda()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

ts = TaskSwitch()
optim = torch.optim.Adam(ts.parameters(), lr=5e-4, weight_decay=1e-4)

nepochs = 2000
pperiod = 100

rest_t = torch.from_numpy(rest).float().cuda()
nback_t = torch.from_numpy(nback).float().cuda()
emoid_t = torch.from_numpy(emoid).float().cuda()

for e in range(nepochs):
    optim.zero_grad()
    xhat = ts(emoid_t[:800])
    loss = rmse(xhat, nback_t[:800])
    loss.backward()
    optim.step()
    if e % pperiod == 0 or e == nepochs-1:
        print(f'{e} {pretty(loss)}')

print('Done')

0 0.3701
100 0.1734
200 0.1632
300 0.1588
400 0.1561
500 0.1549
600 0.1535
700 0.1528
800 0.1538
900 0.1506
1000 0.1504
1100 0.1501
1200 0.1491
1300 0.1494
1400 0.1485
1500 0.1483
1600 0.1481
1700 0.1473
1800 0.1474
1900 0.1474
1999 0.1476
Done


In [5]:
# MLP

with torch.no_grad():
    xhat = ts(emoid_t[800:])
    avg = []
    for i in range(len(xhat)):
        loss = rmse(xhat[i], nback_t[800+i])
        avg.append(loss.detach().cpu().numpy())
    print(np.mean(avg), np.std(avg))

0.18082082 0.03998015


In [6]:
# Avg

diff = torch.mean(nback_t[:800]-emoid_t[:800], dim=0).unsqueeze(0)
xhat = diff+emoid_t[800:]
avg = []
for i in range(len(xhat)):
    loss = rmse(xhat[i], nback_t[800+i])
    avg.append(loss.detach().cpu().numpy())
print(np.mean(avg), np.std(avg))


0.19489487 0.055324692


In [8]:
# Just same

avg = []
for i in range(len(xhat)):
    loss = rmse(nback_t[800+i], emoid_t[800+i])
    avg.append(loss.detach().cpu().numpy())
print(np.mean(avg), np.std(avg))

0.19761838 0.054033354


In [13]:
# VAE with random samples

with torch.no_grad():
    z = vae.enc(emoid_t[800:])
    ag = torch.from_numpy(age[800:]).float().cuda()
    sx = torch.from_numpy(sex[800:]).float().cuda()
    rc = torch.from_numpy(race[800:]).float().cuda()
    tsk = torch.ones(len(z)).float().cuda()
    avg = []
    avgavg = []
    for i in range(len(z)):
        min = 100
        for j in range(20):
            zz = vae.gen(1)
            xhat = vae.dec(zz, ag[i:i+1], sx[i:i+1], rc[i:i+1], 0*tsk[i:i+1], 1*tsk[i:i+1], 0*tsk[i:i+1])
            loss = rmse(xhat[0], nback_t[800+i])
            avgavg.append(float(loss))
            if loss < min:
                min = float(loss)
        avg.append(min)
    print(np.mean(avg), np.std(avg))
    print(np.mean(avgavg), np.std(avgavg))

0.1741209114941073 0.027648964295041413
0.22411603765798285 0.05707637291857793
